<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jcc/02-Word2Vec%2BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM+Word2Vec

En esta notebook, se entrena y prueba la clasificación de oraciones usando LSTM y Word2Vec pre-entrenado.

El principal beneficio de la incrustación de palabras es que incluso las palabras que no se ven durante el entrenamiento se pueden predecir bien ya que la incrustación de palabras está pre-entrenada con un conjunto de datos más grande que los del dataset actual.


## Carga de librerías, modelo word2vec pre-entrenado y funciones útiles

### Carga de librerías a utilizar para el entrenamiento del modelo

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

import numpy as np
import pandas as pd
import sys

# Se instala gensim que es el que tiene el modelo Word2Vec
!pip install gensim
import gensim

### Carga del modelo Word2Vec pre-entrenado de Universidad de Chile

Referencias: https://github.com/dccuchile/spanish-word-embeddings

In [2]:
import os.path
from os import path

if not(path.exists("SBW-vectors-300-min5.bin.gz")):
  !wget http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.bin.gz

# Defino el largo del vector de embedding, en el caso que vamos a usar es 300
VECTOR_EMBEDDINGS = 300

from gensim.models import Word2Vec

filename="SBW-vectors-300-min5.bin.gz"
embeddings = gensim.models.KeyedVectors.load_word2vec_format(filename, binary=True)
embeddings.init_sims(replace=True)

### Se cargan funciones que se utilizan para el pre-procesamiento de las secuencias

In [3]:
def get_max_length(text):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in text:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def embed(texts): 
  """
  devuelve un tensor de tensores con los embedding de las palabras
  para usar en la red LSTM
  """
  # Inicializo el tensor principal con un array de (1, 250) de ceros
  tensor_principal = tf.convert_to_tensor(np.zeros((1, VECTOR_EMBEDDINGS)), dtype=tf.float32)
  iteration = 0
  for word in texts:  
    try:
      e = tf.convert_to_tensor(np.reshape(embeddings.get_vector(word), (1, VECTOR_EMBEDDINGS)), dtype=tf.float32)
    except:
      e = tf.convert_to_tensor(np.zeros((1, VECTOR_EMBEDDINGS)), dtype=tf.float32)
    if iteration==0:
      tensor_principal = e
    else:
      tensor_principal = tf.concat([tensor_principal, e], 0)
    iteration = iteration+1
    
  return tensor_principal


def get_word2vec_enc(texts):
    """
    recibe todos los textos y devuelve una array numpy de tensores de correos
    cada tensor tiene tensores con el embedding de cada palabras
    """
    encoded_texts = []
    for text in texts:
        tokens = text.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_texts.append(word2vec_embedding)
    return encoded_texts
        
def get_padded_encoded_text(encoded_text, max_length):
    """
    para frases cortas se rellena con ceros el array a efectos de contar con la
    misma longitud
    """
    padded_text_encoding = []
    for enc_text in encoded_text:

        zero_padding_cnt = max_length - enc_text.shape[0]
        pad = np.zeros((1, VECTOR_EMBEDDINGS))
        for i in range(zero_padding_cnt):
            enc_text = np.concatenate((pad, enc_text), axis=0)
        padded_text_encoding.append(enc_text)
    return padded_text_encoding

def category_encode(category):
    """
    Se encodea la clase en variables dummies
    """
    return pd.get_dummies(category)


def preprocess(x, y, max_length):
    """
    se encodean x e y llamando a las funciones get_word2_vec_enc, 
    get_padded_encoded_text y category_encode
    """
    # encode words into word2vec
    text = x.tolist()
    
    encoded_text = get_word2vec_enc(text)
    padded_encoded_text = get_padded_encoded_text(encoded_text, max_length)
    
    # encoded class
    categorys = y.tolist()
    encoded_category = category_encode(categorys)
    X = np.array(padded_encoded_text)
    Y = np.array(encoded_category)
    return X, Y 

## Carga del dataset y balanceo de clases

In [4]:
def get_clases():
  '''
  Esta función retorna las etiquetas de las clases sobre el total de los correos. 
  Tomado de notebooks/jcc/de train_test_data.ipynb
  '''
  import numpy as np

  etiquetas_clases = np.array(['Boleto Universitario', 
                               'Cambio de Carrera', 
                               'Cambio de Comisión',
                               'Carga de Notas', 
                               'Certificados Web', 
                               'Consulta por Equivalencias',
                               'Consulta por Legajo', 
                               'Consulta sobre Título Universitario',
                               'Cursadas', 
                               'Datos Personales', 
                               'Exámenes',
                               'Ingreso a la Universidad',
                               'Inscripción a Cursadas',
                               'Pedido de Certificados',
                               'Problemas con la Clave',
                               'Reincorporación', 
                               'Requisitos de Ingreso',
                               'Simultaneidad de Carreras', 
                               'Situación Académica',
                               'Vacunas Enfermería'])

  return etiquetas_clases


def cargar_dataset(URL_data, file_train, file_test, nombre_clase, class_labels, cantidad_clases, texto_otras, origen_ds):
  '''
  Carga los train y test set y genera la reducción de clases, en caso que sea necesario
  '''
  import pandas as pd
  import numpy as np

  # Genero el enlace completo
  URL_file_train = URL_data + file_train
  URL_file_test = URL_data + file_test
  
  # Me traigo los archivos de train y test
  if origen_ds == 'WEB':  
    import wget
    wget.download(URL_file_train)
    wget.download(URL_file_test)
  else:
    file_train = URL_file_train
    file_test = URL_file_test
    
  # Leemos el archivo en un dataframe
  df_train = pd.read_csv(file_train)
  df_test = pd.read_csv(file_test)

  # Agrupamiento de clases
  # Se realiza un conteo de frecuencia por clase y se toman los correos que pertenecen a 
  # las N-cantidad_clases menos observadas
  clases = df_train.clase.value_counts()
  clases_minoritarias = clases.iloc[cantidad_clases-1:].keys().to_list()

  # Agrego a las etiquetas la etiqueta "Otras Consultas" para el agrupamiento
  etiquetas_clases = np.append(class_labels, texto_otras)

  # Genero una nueva clave de clases para "Otras Consultas" a modo de agrupar las que poseen menos apariciones
  df_train.clase[df_train[nombre_clase].isin(clases_minoritarias)] = np.where(etiquetas_clases == texto_otras)[0]
  df_test.clase[df_test[nombre_clase].isin(clases_minoritarias)] = np.where(etiquetas_clases == texto_otras)[0]

  print("El conjunto de entrenamiento tiene la dimensión: " + str(df_train.shape))
  print("El conjunto de testeo tiene la dimensión: " + str(df_test.shape))

  return df_train, df_test


def df_x_y_rn(df, atributo_consulta, atributo_clase):
  '''
  Función para separar en x e y
  '''
  import pandas as pd
  
  # Separo en x e y -train-
  y = df[atributo_clase].to_numpy()
  x = df[atributo_consulta]

  return x, y

In [5]:
import warnings
warnings.filterwarnings("ignore")

# Defino la cantidad de clases
CANTIDAD_CLASES = 4

# Cargo el dataset
df_train, df_test = cargar_dataset('https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/consolidado_jcc/', 'correos-train-80.csv', 'correos-test-20.csv', 'clase', get_clases(), CANTIDAD_CLASES, "Otras Consultas", 'COLAB')

# Separo en train y test
x_train, y_train = df_x_y_rn(df_train, 'Consulta', 'clase')
x_test, y_test = df_x_y_rn(df_test, 'Consulta', 'clase')

El conjunto de entrenamiento tiene la dimensión: (800, 24)
El conjunto de testeo tiene la dimensión: (200, 24)


## Preprocesamiento (codificación del texto a vectores numéricos)

In [6]:
# Largo máximo de las consultas
max_length = get_max_length(x_train)

# Preproceso las secuencias de texto
x_train_processed, y_train_processed = preprocess(x_train, y_train, max_length)
x_test_processed, y_test_processed = preprocess(x_test, y_test, max_length)

## Construcción del Modelo

In [7]:
# LSTM model
model = Sequential()
model.add(LSTM(32))
model.add(Dense(CANTIDAD_CLASES, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Entrenamiento del modelo

In [8]:
print('Train...')
model.fit(x_train_processed, y_train_processed, epochs=20)

Train...
Epoch 1/20
25/25 [==============================] - 3s 35ms/step - loss: 1.3661 - accuracy: 0.3290
Epoch 2/20
25/25 [==============================] - 1s 35ms/step - loss: 1.3065 - accuracy: 0.4022
Epoch 3/20
25/25 [==============================] - 1s 34ms/step - loss: 1.2121 - accuracy: 0.4638
Epoch 4/20
25/25 [==============================] - 1s 34ms/step - loss: 1.0670 - accuracy: 0.5373
Epoch 5/20
25/25 [==============================] - 1s 35ms/step - loss: 1.0618 - accuracy: 0.5770
Epoch 6/20
25/25 [==============================] - 1s 35ms/step - loss: 0.9635 - accuracy: 0.6127
Epoch 7/20
25/25 [==============================] - 1s 34ms/step - loss: 0.8663 - accuracy: 0.6896
Epoch 8/20
25/25 [==============================] - 1s 34ms/step - loss: 0.8575 - accuracy: 0.6815
Epoch 9/20
25/25 [==============================] - 1s 33ms/step - loss: 0.8382 - accuracy: 0.6549
Epoch 10/20
25/25 [==============================] - 1s 34ms/step - loss: 0.7540 - accuracy: 0.7150


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (32, 32)                  42624     
_________________________________________________________________
dense (Dense)                (32, 4)                   132       
Total params: 42,756
Trainable params: 42,756
Non-trainable params: 0
_________________________________________________________________


### Testeo del Modelo

In [10]:
score, acc = model.evaluate(x_test_processed, y_test_processed, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

7/7 - 1s - loss: 0.7358 - accuracy: 0.7250
Test score: 0.7358402013778687
Test accuracy: 0.7250000238418579
